In [ ]:
!aws s3 cp s3://openproblems-bio/public/neurips-2023-competition/sc_counts.h5ad  ./resources_raw/ --no-sign-request
aws s3 cp s3://openproblems-bio/public/neurips-2023-competition/2023-09-14_kaggle_upload/2023-08-31_sc_multiome_expression_atac.h5ad ./resources_raw/ --no-sign-request

In [ ]:
# !pip install sctk, anndata

import anndata as ad 
import pandas as pd
import numpy as np
import sctk
par = {
    'sc_counts': '../resources_raw/sc_counts.h5ad',
    'sc_counts_filtered': '../resources_raw/sc_counts_filtered.h5ad',
}
def preprocess_sc(par):
    # clean up
    sc_counts = ad.read_h5ad(par['sc_counts'])
    sc_counts.obs = sc_counts.obs[['well', 'row', 'col', 'plate_name', 'cell_type', 'donor_id']]
    sc_counts.X = sc_counts.layers['counts']
    del sc_counts.layers 
    del sc_counts.obsm 
    sc_counts.var_names_make_unique()
    sc_counts.obs['plate_well_cell_type'] = sc_counts.obs['plate_name'].astype('str') \
        + '_' + sc_counts.obs['well'].astype('str') \
        + '_' + sc_counts.obs['cell_type'].astype('str')
    sc_counts.obs['plate_well_cell_type'] = sc_counts.obs['plate_well_cell_type'].astype('category')

    # merge cell types
    CELL_TYPES = ['NK cells', 'T cells CD4+', 'T cells CD8+', 'T regulatory cells', 'B cells', 'Myeloid cells']
    T_cell_types = ['T regulatory cells', 'T cells CD8+', 'T cells CD4+']
    cell_type_map = {cell_type: 'T cells' if cell_type in T_cell_types else cell_type for cell_type in CELL_TYPES}
    sc_counts.obs['cell_type'] = sc_counts.obs['cell_type'].map(cell_type_map)
    sc_counts.obs['cell_type'].unique()

    # qc 
    sctk.calculate_qc(sc_counts)
    sctk.cellwise_qc(sc_counts)

    # filtering
    # cell wise
    filter_percent_hb = sc_counts.obs.percent_hb>.2
    filter_percent_hb.sum()
    # gene wise
    plates = sc_counts.obs['plate_name'].unique()

    # Step 2: Initialize a DataFrame to store counts
    gene_counts_per_plate = pd.DataFrame(index=sc_counts.var_names, columns=plates, dtype=int)

    # Step 3: Iterate over each plate and calculate expression counts
    for plate in plates:
        # Subset the AnnData object for the current plate
        subset = sc_counts[sc_counts.obs['plate_name'] == plate]

        # Calculate expression counts (genes x cells > 0)
        expressed_genes = (subset.X > 0).sum(axis=0)

        # Check if the result needs conversion from sparse matrix format
        if isinstance(expressed_genes, np.matrix):
            expressed_genes = np.array(expressed_genes).flatten()

        # Store the counts in the DataFrame
        gene_counts_per_plate[plate] = expressed_genes

    # Step 4: Aggregate counts across plates (max or sum based on the requirement)
    # We use `max` here to find if any gene meets the criteria in at least one plate
    max_counts = gene_counts_per_plate.max(axis=1)

    # Step 5: Create a mask for genes to keep (genes expressed in at least 100 cells in any plate)
    genes_to_keep = max_counts >= 100
    print('retained genes:', genes_to_keep.sum())
    # actual filtering
    sc_counts = sc_counts[(~filter_percent_hb), genes_to_keep]
    # clean
    sc_counts.obs = sc_counts.obs[['cell_type', 'sm_name', 'donor_id', 'row', 'plate_name', 'well']]
    sc_counts.var = sc_counts.var[[]]

    del sc_counts.obsm
    del sc_counts.uns

    sc_counts.write(par['sc_counts_filtered'])
preprocess_sc(par)

In [ ]:
import anndata as ad 
from scipy import sparse
par = {
    'sc_counts_filtered': '../resources_raw/sc_counts_filtered.h5ad',
    'pseudobulked': '../resources_raw/pseudobulked.h5ad',
}
def sum_by(adata: ad.AnnData, col: str) -> ad.AnnData:
    """
    Adapted from this forum post: 
    https://discourse.scverse.org/t/group-sum-rows-based-on-jobs-feature/371/4
    """
    
    assert pd.api.types.is_categorical_dtype(adata.obs[col])

    # sum `.X` entries for each unique value in `col`
    cat = adata.obs[col].values

    indicator = sparse.coo_matrix(
        (
            np.broadcast_to(True, adata.n_obs),
            (cat.codes, np.arange(adata.n_obs))
        ),
        shape=(len(cat.categories), adata.n_obs),
    )
  
    sum_adata = ad.AnnData(
        indicator @ adata.X,
        var=adata.var,
        obs=pd.DataFrame(index=cat.categories),
    )
    
    # copy over `.obs` values that have a one-to-one-mapping with `.obs[col]`
    obs_cols = adata.obs.columns
    obs_cols = list(set(adata.obs.columns) - set([col]))
    
    one_to_one_mapped_obs_cols = []
    nunique_in_col = adata.obs[col].nunique()
    for other_col in obs_cols:
        if len(adata.obs[[col, other_col]].drop_duplicates()) == nunique_in_col:
            one_to_one_mapped_obs_cols.append(other_col)

    joining_df = adata.obs[[col] + one_to_one_mapped_obs_cols].drop_duplicates().set_index(col)
    assert (sum_adata.obs.index == sum_adata.obs.join(joining_df).index).all()
    sum_adata.obs = sum_adata.obs.join(joining_df)
    sum_adata.obs.index.name = col
    sum_adata.obs = sum_adata.obs.reset_index()
    sum_adata.obs.index = sum_adata.obs.index.astype('str')

    return sum_adata
def pseudobulked_and_filter(par):
    # pseudobulk
    sc_counts = ad.read_h5ad(par['sc_counts_filtered'])
    bulk_adata = sum_by(sc_counts, 'plate_well_cell_type')
    bulk_adata.obs['cell_count'] = sc_counts.obs.groupby('plate_well_cell_type').size().values
    bulk_adata.X = np.array(bulk_adata.X.todense())

    print('ratio of missingness' , (bulk_adata.X==0).sum()/bulk_adata.X.size)
    bulk_adata.var = bulk_adata.var.reset_index()
    bulk_adata.var.set_index('index', inplace=True)

    bulk_adata.X = np.nan_to_num(bulk_adata.X, nan=0)

    
